# Libraries

In [133]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq
import re
import time
from collections import defaultdict
import operator
import math
import nltk
from nltk.tokenize import word_tokenize
import os
import matplotlib.pyplot as plt
from csv import writer
import unicodedata

# Functions

In [229]:
def getdoc(url):
    items = uReq(url)
    soup = bs(items, 'html.parser')
    return soup

In [228]:
def getCitiesNameUrl(soupobject):
    citiestable = soupobject.find('table', {'class': 'wikitable sortable'})
    citiesname = []
    citiesurl = []
    for i in range(1,len(citiestable.findAll('tr'))):
        a = (citiestable.findAll('tr'))[i]
        a = a.findAll('td')
        a = a[1].find('a', attrs={'href': re.compile("^/wiki/")})
        citiesname.append(a.text)
        citiesurl.append("https://en.wikipedia.org" + a.get('href'))
    return citiesname, citiesurl

In [227]:
def getCityDetail(citiesname, citiesurl):
    
    maindf = pd.DataFrame()
    maindf['Name'] = np.nan
    
    for k in range(len(citiesname)):
        city = citiesurl[k]
        cityinfo = getdoc(city)
        cityfields = cityinfo.find('table', {'class': 'infobox geography vcard'})
        counter = 0
        
        for i in range(1, len(cityfields.findAll('tr'))):
            if (len(cityfields.findAll('tr')[i].findAll('th')) > 0):
                counter = i
                break    
                
        citydetail = {}
        
        for i in range(counter, len(cityfields.findAll('tr'))):
            attribute = cityfields.findAll('tr')[i].findAll('th')[0].text
            if (len(cityfields.findAll('tr')[i].findAll('td')) > 0):
                attributeValue = cityfields.findAll('tr')[i].findAll('td')[0].text
                citydetail[unicodedata.normalize("NFKC", attribute)] = unicodedata.normalize("NFKC", attributeValue)
            if(attribute == "Website"):
                break
        
        maindf.loc[k, 'Name'] = citiesname[k] 

        for i,j in citydetail.items():
            if i not in maindf.columns:
                maindf[i] = np.nan
                maindf.loc[k, i] = j
            if i in maindf.columns:
                maindf.loc[k, i] = j
                
    return maindf

# Main Function

## Getting List of Cities

In [230]:
citieslisturl = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
citieslistpage = getdoc(citieslisturl)

In [231]:
citiesname, citiesurl = getCitiesNameUrl(citieslistpage)

## Getting Table of Fields of Cities

In [232]:
cityTabledf = getCityDetail(citiesname, citiesurl)

# Writing To CSV

In [ ]:
cityTabledf.to_csv("Cities_Detail.csv")